In [ ]:
import requests
import json
import os
from instabase.notebook.conf import ibconfig

# URI
INSTABASE_URI = 'https://www.instabase.com'
RUN_FLOW_API = INSTABASE_URI + '/api/v1/flow/run_flow_async'
GET_FLOW_STATUS_API = INSTABASE_URI + '/api/v1/jobs/status'
RUN_METAFLOW_API = INSTABASE_URI + '/api/v1/flow/run_metaflow_async'

# Request header
MY_API_TOKEN = '<MY_API_TOKEN>'                                         # Fill this out: Put your API token here.
HEADERS = {
  'Authorization': 'Bearer {}'.format(MY_API_TOKEN),
}

def _link(path):
  s = INSTABASE_URI + path
  return s.replace(' ', '%20')

def _pretty_print(d):
  try:
    d = json.loads(d)
  except:
    pass
  return json.dumps(d, sort_keys=True, indent=4, separators=(',', ': '))

def run_metaflow(input_dir_path, flow_dir_path, classifier_file_path):
  data = json.dumps({
    'input_dir': input_dir_path,
    'flow_root_dir': flow_dir_path,
    'classifier_file_path': classifier_file_path,
    'delete_out_dir': True,
  })
  r = requests.post(RUN_METAFLOW_API, headers=HEADERS, data=data)
  return r.content

project_root_path = '<PATH_TO_PROJECT>'                                     # Fill this out.
input_dir_path = '{}/<PATH_TO_INPUT>'.format(project_root_path)             # Fill this out.
flow_dir_path = '{}/<PATH_TO_FLOWS>'.format(project_root_path)              # Fill this out.
classifier_file_path = '{}/<PATH_TO_CLASSIFIER>'.format(project_root_path)  # Fill this out.

metaflow_resp = run_metaflow(input_dir_path, flow_dir_path, classifier_file_path)
print(_pretty_print(metaflow_resp))

metaflow_resp_dict = json.loads(metaflow_resp)

In [ ]:
import StringIO
import time

def get_status(job_id):
    r = requests.get(
        GET_FLOW_STATUS_API + '?job_id=' + job_id, headers=HEADERS, cookies=ibconfig.cookies)
    print(r.content)
    return json.loads(r.content)

def _display_file(full_path):
    
    _, ext = os.path.splitext(full_path)
    
    with ib.open('/' + full_path.lstrip('/'), 'r') as f:
        content = f.read()
    
    if ext == '.csv':
        display(pd.read_csv(StringIO.StringIO(content)))
    else:
        print(content)
    
def print_metaflow_result(flow_resp):
    
    job_id = flow_resp['data']['job_id']
    
    print('Showing flow results for job: ' +job_id)
    done = False
    resp = {}
    
    while not done:
        resp = get_status(job_id)
        print(resp)
        if resp['status'] != 'OK':
            print(resp['msg'])
            return

        if resp['state'] != 'DONE':
            cur_status = json.loads(resp.get('cur_status', '{}'))
            step_name = cur_status.get('stepName')
            msg = cur_status.get('curMsg')
            print('Running: ' + step_name + ', ' + msg + '...')
            time.sleep(4)
            continue
        
        done = True
        break
    
    classified_groups_file = resp['results'][0]['groups_file']
    output_folders_file = resp['results'][0]['class_output_file']
    
    print('\n=== Summaries ===')
    print('\n= Classified Groups =')
    _display_file(classified_groups_file)
    
    print('\n= Final Output Folders =')
    _display_file(output_folders_file)



print_metaflow_result(metaflow_resp_dict)